In [175]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
import json
from tqdm import tqdm
from os.path import join,isfile
from os import listdir

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [207]:
ua = UserAgent()
bad_requests = []

def execute_query(qquery, entity_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity_id, f = 0):
        res = []
        if f > 5:
            global bad_requests
            bad_requests.append(entity_id)
            print('Bad request ', entity_id)
            return None
        try:
            results= execute_query(bookquery.subclass_with_parents, entity_id)
            for result in results:
                entities = {}
                entities['subclass'] = result['show']['value'].split('/')[-1]
                entities['subclasslabel'] = result['showLabel']['value']
                entities['classid'] = result['parent']['value'].split('/')[-1]
                entities['classlabel'] = result['parentLabel']['value']
                res.append(entities)
            return pd.DataFrame(data = res, columns = ['classid', 'classlabel' ,'subclass', 'subclasslabel'])
        except Exception as e:
            print(e)
            time.sleep(5)
            print('Try again ', entity_id, f)
            return entities_sparqul(entity_id, f + 1)
        
# def getting_subclass(subclasses , name):
#     main_df = pd.DataFrame()
#     for item in subclasses:
#         df = entities_sparqul("Q4022")
#         df = df[df['classid'].isin(list(df.subclass))]
#         df =  df[lambda x : x['subclasslabel'].str[0]!='Q']
#         new_df = df.groupby(['subclass' , 'subclasslabel']).apply(lambda x: ','.join(list(map(lambda x: '_'.join(x) , zip(x.classid,x.classlabel))))).reset_index()
#         new_df.columns = ['subclass' , 'subclasslabel' , 'class']
#         new_df['signifance'] = ''
#         main_df = pd.concat([main_df , new_df])
#     main_df.to_excel(f'excel_with_subclasses/{name}.xlsx' , index = False)
    
def new_sheets(sheetname , ID , filename):
    print(sheetname , filename)
    df = entities_sparqul(ID)
    df = df[df['classid'].isin(list(df.subclass))]
    df =  df[lambda x : x['subclasslabel'].str[0]!='Q']
    if len(df) == 0:
        return
    new_df = df.groupby(['subclass' , 'subclasslabel']).apply(lambda x: ','.join(list(map(lambda x: '_'.join(x) , zip(x.classid,x.classlabel))))).reset_index()
    new_df.columns = ['subclass' , 'subclasslabel' , 'class']
    new_df['signifance'] = ''
    if sheetname == filename or filename not in list(map(lambda x: x.split("\\")[1].split('.')[0] , counter_for_files("excel_with_subclasses"))):
        new_df.to_excel(f'excel_with_subclasses/{filename}.xlsx', sheet_name = sheetname ,encoding='utf-8-sig' , index = False)
    else:
        with pd.ExcelWriter(f'excel_with_subclasses/{filename}.xlsx' ,  engine="openpyxl" , mode  = 'a') as writer:
            new_df.to_excel(writer, sheet_name = sheetname , encoding='utf-8-sig' , index = False)
        
def new_file(name , dic):
    if name not in list(map(lambda x: x.split("\\")[1].split('.')[0] , counter_for_files("excel_with_subclasses"))):
        for k,v in dic.items():
            if k == name:
                new_sheets(k,v , name)
                del(dic[k])
                break
        for k,v in dic.items():
            new_sheets(k,v , name)

In [208]:
def subclass_json(k,v):
    new_d = {}
    for item in v:
        if item[0] == 'Q':
            new_d[k.split(':')[1]] = item
        elif item in d2.keys():
            new_d[item.split(':')[1]] = d2[item][0]
        else:
            subclass_json(item , d[item])
    return new_d

In [209]:
arr = []
for k,v in d.items():
    arr.append((subclass_json(k,v) , k.split(":")[1]))

In [210]:
arr

[({'port': 'Q44782', 'seaport': 'Q15310171'}, 'port'),
 ({'road': 'Q34442',
   'highway': 'Q269949',
   'controlled_access_highway': 'Q46622',
   'aerial_lift': 'Q13409250'},
  'road'),
 ({'tunnel': 'Q44377',
   'road_tunnel': 'Q2354973',
   'railway_tunnel': 'Q1311958'},
  'tunnel'),
 ({'bridge': 'Q12280',
   'arch_bridge': 'Q158438',
   'suspension_bridge': 'Q12570',
   'moveable_bridge': 'Q787417'},
  'bridge'),
 ({'pyramid': 'Q12516', 'egyptian_pyramids': 'Q175582'}, 'pyramid'),
 ({'building': 'Q41176',
   'skyscraper': 'Q11303',
   'house': 'Q3947',
   'castle': 'Q23413'},
  'building'),
 ({'architectural_structure': 'Q811979',
   'tower': 'Q12518',
   'statue': 'Q179700',
   'monument': 'Q4989906',
   'transmitter_mast': 'Q1068623',
   'lighthouse': 'Q39715',
   'cathedral': 'Q2977',
   'church_building': 'Q16970',
   'mosque': 'Q32815',
   'museum': 'Q33506',
   'theatre': 'Q24354',
   'airport': 'Q1248784',
   'square': 'Q174782',
   'swimming_pool': 'Q1501',
   'well': 'Q43483

In [211]:
for item in tqdm(arr):
    new_file(item[1] , item[0])





  0%|                                                                                           | 0/34 [00:00<?, ?it/s]

state_of_the_united_states state
state_of_india state
state_of_australia state
state_of_venezuela state
state_or_territory_of_australia state
federated_state state






 50%|█████████████████████████████████████████                                         | 17/34 [00:03<00:03,  5.42it/s]

republic republic
republic_of_russia republic






 53%|███████████████████████████████████████████▍                                      | 18/34 [00:04<00:08,  1.87it/s]

...





 94%|█████████████████████████████████████████████████████████████████████████████▏    | 32/34 [01:01<00:03,  1.74s/it]

volcano volcano
active_volcano volcano
dormant_volcano volcano
extinct_volcano volcano






 97%|███████████████████████████████████████████████████████████████████████████████▌  | 33/34 [01:02<00:01,  1.68s/it]

mountain mountain
massif mountain
mountain_range mountain
sierra mountain






100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [01:04<00:00,  1.90s/it]


In [212]:
arr

[({'port': 'Q44782', 'seaport': 'Q15310171'}, 'port'),
 ({'road': 'Q34442',
   'highway': 'Q269949',
   'controlled_access_highway': 'Q46622',
   'aerial_lift': 'Q13409250'},
  'road'),
 ({'tunnel': 'Q44377',
   'road_tunnel': 'Q2354973',
   'railway_tunnel': 'Q1311958'},
  'tunnel'),
 ({'bridge': 'Q12280',
   'arch_bridge': 'Q158438',
   'suspension_bridge': 'Q12570',
   'moveable_bridge': 'Q787417'},
  'bridge'),
 ({'pyramid': 'Q12516', 'egyptian_pyramids': 'Q175582'}, 'pyramid'),
 ({'building': 'Q41176',
   'skyscraper': 'Q11303',
   'house': 'Q3947',
   'castle': 'Q23413'},
  'building'),
 ({'architectural_structure': 'Q811979',
   'tower': 'Q12518',
   'statue': 'Q179700',
   'monument': 'Q4989906',
   'transmitter_mast': 'Q1068623',
   'lighthouse': 'Q39715',
   'cathedral': 'Q2977',
   'church_building': 'Q16970',
   'mosque': 'Q32815',
   'museum': 'Q33506',
   'theatre': 'Q24354',
   'airport': 'Q1248784',
   'square': 'Q174782',
   'swimming_pool': 'Q1501',
   'well': 'Q43483

In [40]:
getting_subclass(["Q4022"] , 'Реки')

In [52]:
with open(r'wikidata\artificial_classes\data\artificial_w_singletonsv6.json') as f:
    d = json.load(f)
print(d)

{'dr:port': ['Q44782', 'dr:seaport'], 'dr:road': ['Q34442', 'dr:highway', 'dr:controlled_access_highway', 'dr:aerial_lift'], 'dr:tunnel': ['Q44377', 'dr:road_tunnel', 'dr:railway_tunnel'], 'dr:bridge': ['Q12280', 'dr:arch_bridge', 'dr:suspension_bridge', 'dr:moveable_bridge'], 'dr:pyramid': ['Q12516', 'dr:egyptian_pyramids'], 'dr:building': ['Q41176', 'dr:skyscraper', 'dr:house', 'dr:castle'], 'dr:architectural_structure': ['Q811979', 'dr:tower', 'dr:statue', 'dr:monument', 'dr:transmitter_mast', 'dr:lighthouse', 'dr:cathedral', 'dr:church_building', 'dr:mosque', 'dr:building', 'dr:pyramid', 'dr:bridge', 'dr:museum', 'dr:theatre', 'dr:airport', 'dr:square', 'dr:swimming_pool', 'dr:well', 'dr:stadium', 'dr:piste', 'dr:tomb', 'dr:aerial_lift', 'dr:ferris_wheel', 'dr:pier', 'dr:fountain'], 'dr:river': ['Q4022', 'dr:navigable_river', 'dr:drainage_system', 'dr:tributary'], 'dr:sea': ['Q165', 'dr:mediterranean_sea'], 'dr:bay': ['Q39594', 'dr:fjord', 'dr:lagoon', 'dr:liman'], 'dr:canal': ['Q1

In [53]:
with open(r'wikidata\artificial_classes\data\singletons_v6.json') as f:
    d2 = json.load(f)

dict_keys(['dr:federal_district', 'dr:liman', 'dr:mosque', 'dr:economic_region', 'dr:state_or_territory_of_australia', 'dr:sinkhole', 'dr:arch_bridge', 'dr:mine', 'dr:slope', 'dr:salt_lake', 'dr:tributary', 'dr:mountain_range', 'dr:typhoon', 'dr:federated_state', 'dr:city_with_millions_of_inhabitants', 'dr:dam', 'dr:spa_town', 'dr:geographical_object', 'dr:suspension_bridge', 'dr:theatre', 'dr:prospekt', 'dr:coral_reef', 'dr:federal_district_of_russia', 'dr:promenade', 'dr:emirate', 'dr:garden_square', 'dr:desert', 'dr:kingdom', 'dr:continental_area_and_surrounding_islands', 'dr:flight', 'dr:coast', 'dr:geographic_location', 'dr:fountain', 'dr:underground_mine', 'dr:drainage_system', 'dr:republic_of_russia', 'dr:quarry', 'dr:administrative_centre', 'dr:hemisphere_of_the_earth', 'dr:volcanic_crater_lake', 'dr:cliff', 'dr:capital', 'dr:oasis', 'dr:cave', 'dr:city_state', 'dr:microstate', 'dr:reef', 'dr:waterfall', 'dr:island_nation', 'dr:tower', 'dr:piste', 'dr:cathedral', 'dr:federal_ci

In [ ]:
new_arr = []
for k , v in d:
    for item in v:
        if v[0] = 'Q':